In [1]:
import os

In [2]:
%pwd

'c:\\Users\\anjik\\Desktop\\MLOPs_projects\\Chest_Disease_Image_Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\anjik\\Desktop\\MLOPs_projects\\Chest_Disease_Image_Classification'

# imports

In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
from cnnClassifier import logger
import tensorflow as tf

from cnnClassifier.utils.common import save_json

# set mlflow tracking uri

In [6]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/malleswarigelli/Chest_Disease_Image_Classification_.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="malleswarigelli"
os.environ["MLFLOW_TRACKING_PASSWORD"]="2285aa424e395caab6840e555b7d9680a386cde7"

In [7]:
# load trained model
import tensorflow as tf
#model = tf.keras.models.load_model("artifacts/model_training/model.h5")
#model

In [8]:
# define config_entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    path_of_trained_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_batch_size: int    
    params_image_size: list

In [9]:
# write configuration manager
class ConfigurationManager:
    """
    ConfigurationManager class captures & returns configuration for components implementation
        
    """
    def __init__(self,
        # params: config.yaml, params.yaml file paths          
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        # read config, params yaml file
        self.config = read_yaml(config_filepath) # returns ConfigBox to access data easily
        self.params = read_yaml(params_filepath)
        
        # create artifact directory
        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config (self) -> ModelEvaluationConfig:
        """
        Method: get_model_evaluation_config
        Params:
        Returns: configuration for Model Evaluation component i.e ModelEvaluationConfig 
        """
        logger.info("Entering get_model_evaluation_config method of ConfigurationManager")
        model_evaluation_config = ModelEvaluationConfig(path_of_trained_model= "artifacts/model_training/model.h5", 
                                                        training_data= "artifacts/data_ingestion/Chest-CT-Scan-data",                                                         
                                                        mlflow_uri= "https://dagshub.com/malleswarigelli/Chest_Disease_Image_Classification_.mlflow", 
                                                        all_params= self.params, 
                                                        params_batch_size= self.params.BATCH_SIZE, 
                                                        params_image_size= self.params.IMAGE_SIZE
            
        )
        logger.info("Exiting get_model_evaluation_config method of ConfigurationManager")
        return model_evaluation_config

In [10]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [11]:
class ModelEvaluation:
    """
     This class tracks model evaluation metrics
    """
    def __init__(self, evaluation_config:ModelEvaluationConfig):
        self.evaluation_config= evaluation_config 
        
    
    def _valid_generator(self):
        """
        Method Name : _valid_generator
        Description : performs split of training data, slice 30% data for validation 
        Output      : 
        On Failure  :   Write an exception log and then raise an exception
        
        """
        logger.info("Data split for validation data started!")
        data_generator_kwargs= dict(rescale= 1./255,
                                   validation_split= 0.30)
        
        dataflow_kwargs= dict(target_size= self.evaluation_config.params_image_size[:-1],
                              batch_size= self.evaluation_config.params_batch_size,
                              interpolation= "bilinear")
        
        valid_data_generator= tf.keras.preprocessing.image.ImageDataGenerator(**data_generator_kwargs)
        self.valid_generator= valid_data_generator.flow_from_directory(directory= self.evaluation_config.training_data,
                                                                       subset= "validation",
                                                                       shuffle= False,
                                                                       **dataflow_kwargs)
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
        
    def evaluation(self):
        """
        Method Name : evaluation
        Description : performs model evaluation
        Output      : 
        On Failure  :   Write an exception log and then raise an exception
        
        """
        logger.info("Model evaluation started!")
        logger.info("Loading the Trained model")
        self.model= self.load_model(self.evaluation_config.path_of_trained_model)
        logger.info("Generating validation data")
        self._valid_generator()
        logger.info("Evaluating the model, saving loss and accuracy scores")
        self.metric= model.evaluate(self.valid_generator)
        self.save_metrics()
        
    def save_metrics(self):
        """
        Method Name : save_score
        Description : saves loss and accuracy scores
        Output      : 
        On Failure  : Write an exception log and then raise an exception
        """
        metrics= {"loss": self.metric[0], "accuracy": self.metric[1]}
        save_json(path= Path("scores.json"), data= metrics)        
        
        
    def log_into_mlflow(self):
        """
        Method Name : log_into_mlflow
        Description : log experiments in mlflow
        Output      : 
        On Failure  : Write an exception log and then raise an exception
        """
        logger.info("Logging Experiments stated in mlflow")
        mlflow.set_registry_uri(self.evaluation_config.mlflow_uri)
        tracking_url_type_store= urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            logger.info("Logging parameters metrics")
            mlflow.log_params(self.evaluation_config.all_params)
            mlflow.log_metrics(
                {"loss":self.metric[0],
                 "accuracy":self.metric[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(self.model, "model", registered_model_name="Vgg16Model")
            else:
                mlflow.sklearn.log_model(self.model, "model")
            
        
        
        

In [12]:
try:
    logger.info("ModelEvaluation component started")
    logger.info("Loading of ModelEvaluation component configuration started")
    config = ConfigurationManager() # create object for ConfigurationManager class
    evaluation_config= config.get_model_evaluation_config() # obj.method() returns ModelEvaluationConfig
    logger.info("All configuration directories, files needed for ModelEvaluation component are ready")
    
    logger.info("ModelEvaluation steps started")
    model_evaluation = ModelEvaluation(evaluation_config=evaluation_config) # create object for ModelTraining class
    model_evaluation.evaluation() # obj.method()    
    model_evaluation.log_into_mlflow()

    logger.info("OK! ModelEvaluation component completed")
except Exception as e:
    raise e

[2024-03-17 19:05:58,038: INFO: 3925082432: ModelEvaluation component started]
[2024-03-17 19:05:58,039: INFO: 3925082432: Loading of ModelEvaluation component configuration started]
[2024-03-17 19:05:58,042: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-17 19:05:58,044: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-17 19:05:58,046: INFO: common: created directory at: artifacts]
[2024-03-17 19:05:58,046: INFO: 3105106954: Entering get_model_evaluation_config method of ConfigurationManager]
[2024-03-17 19:05:58,047: INFO: 3105106954: Exiting get_model_evaluation_config method of ConfigurationManager]
[2024-03-17 19:05:58,048: INFO: 3925082432: All configuration directories, files needed for ModelEvaluation component are ready]
[2024-03-17 19:05:58,048: INFO: 3925082432: ModelEvaluation steps started]
[2024-03-17 19:05:58,049: INFO: 1091401809: Model evaluation started!]
[2024-03-17 19:05:58,050: INFO: 1091401809: Loading the Trained model

NameError: name 'model' is not defined